In [1]:
data_var = '2024-01-21'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
12587,2024-01-21,Espanha Acb,08:30,Breogan,Basquet Girona,1.53,2.53,160.5,1.80,1.94,-5.5,2.00,1.77,QHQMHZdJ,0.653595,0.395257,0.555556,0.515464,0.048852,0.8,0.0,NaN,NaN,260.084,103.239369,0.396946,194.426,69.026762,0.355028,190.182,0.000,149.60,227.92,0.0,0.0,0.0,0.0,0.348328,0.052938,0.086278,-1.73,-0.346,-1.531792,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
12588,2024-01-21,Espanha Acb,08:30,Granada,Barcelona,3.63,1.30,164.5,1.81,1.95,6.5,2.06,1.69,x6X7uwkQ,0.275482,0.769231,0.552486,0.512821,0.044713,0.0,0.4,NaN,NaN,333.968,204.573943,0.612556,111.014,16.982649,0.152978,0.000,115.574,123.97,132.80,0.0,0.0,0.0,0.0,0.668381,0.052657,0.139536,0.00,0.000,inf,0.0,0.0,0.0,-2.23,-0.446,-0.672646,0.0,0.0,0.0
12589,2024-01-21,Espanha Acb,13:00,Baskonia,MoraBanc Andorra,1.27,3.83,170.5,1.80,1.96,-10.5,2.08,1.68,vLE4tJZJ,0.787402,0.261097,0.555556,0.510204,0.048498,0.8,0.6,NaN,NaN,151.730,42.076036,0.277309,228.976,91.347564,0.398939,131.884,334.476,170.00,374.74,0.0,0.0,0.0,0.0,0.709880,0.060179,0.150448,0.25,0.050,5.400000,0.0,0.0,0.0,-5.00,-1.000,-2.830000,0.0,0.0,0.0
12590,2024-01-21,Espanha Acb,13:00,Zaragoza,Valencia,2.14,1.72,157.5,1.82,1.96,1.5,1.96,1.80,OY7a804J,0.467290,0.581395,0.549451,0.510204,0.048685,0.4,0.4,NaN,NaN,254.436,126.642915,0.497740,166.864,140.638651,0.842834,213.810,213.100,111.78,418.10,0.0,0.0,0.0,0.0,0.153878,0.052378,0.060179,4.94,0.988,1.153846,0.0,0.0,0.0,-1.16,-0.232,-3.103448,0.0,0.0,0.0
12591,2024-01-21,Espanha Acb,14:30,Real Madrid,Bilbao,1.04,11.00,163.5,1.88,1.88,-17.5,2.06,1.71,AVJHqHRI,0.961538,0.090909,0.531915,0.531915,0.052448,0.2,0.4,NaN,NaN,118.202,39.202236,0.331655,168.250,91.776061,0.545474,102.532,253.682,95.45,195.00,0.0,0.0,0.0,0.0,1.169898,0.000000,0.131293,-0.52,-0.104,-0.384615,0.0,0.0,0.0,0.20,0.040,250.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12779,2024-01-21,Islândia Copa Da Islândia,15:00,Tindastoll,KR Basket,1.02,11.10,172.5,1.84,1.84,-26.5,2.01,1.71,v3chYJRA,0.980392,0.090090,0.543478,0.543478,0.070482,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,1.176178,0.000000,0.114049,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
12780,2024-01-21,Lituânia King Mindaugas Cup,13:30,Zalgiris Kaunas,BC Wolves,1.11,6.15,158.5,1.84,1.86,-13.5,1.97,1.74,0dbpf8hl,0.900901,0.162602,0.543478,0.537634,0.063503,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.981768,0.007644,0.087674,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
12781,2024-01-21,Reino Unido Troféu Da Bbl,15:00,Cheshire Phoenix,Sheffield Sharks,1.36,2.97,163.5,1.83,1.87,-9.5,2.10,1.62,8ltVCunJ,0.735294,0.336700,0.546448,0.534759,0.071994,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,117.30,140.58,0.0,0.0,0.0,0.0,0.525839,0.015289,0.182479,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
12782,2024-01-21,Rússia Superliga,13:00,M. Khimki,Dome-Springs Izhevsk,1.05,8.25,144.5,1.83,1.87,-17.5,2.08,1.63,ncCIbqRD,0.952381,0.121212,0.546448,0.534759,0.073593,0.4,0.0,NaN,NaN,83.944,6.864960,0.081780,139.790,67.652586,0.483959,90.484,0.000,79.50,260.52,0.0,0.0,0.0,0.0,1.094875,0.015289,0.171535,0.72,0.144,0.347222,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,20:00,Eua Nba,Orlando Magic,Miami Heat,213.5,1.82,0.7551,Over
1,20:00,Eua Nba,Washington Wizards,Denver Nuggets,237.5,1.85,0.8720,Over
2,21:00,Eua Nba,Houston Rockets,Boston Celtics,229.5,1.86,0.9685,Over
3,22:00,Eua Nba,Phoenix Suns,Indiana Pacers,245.5,1.87,0.8991,Over
